# Preparation of the UN General Debates Data Set

In order to simplify data access in Chapter 1 we prepared a slightly modified version of the data set where speeches and speakers are stored in a single table resp. file, `un-general-debates-blueprint.csv`. The columns are named consistently and some data errors are resolved.

## Download Data Files

You need to download the source data files from these locations and store them in the same directory as this notebook:

* [Speakers_by_session.csv](https://dataverse.harvard.edu/api/access/datafile/3035579): Download from Harvard
* [un-general-debates.csv](https://www.kaggle.com/unitednations/un-general-debates): Download from Kaggle (login required)

## Addition Requirements

Run

```sh
!pip install pycountry
```

## Prepare Speakers Data

In [ ]:
import pandas as pd

file = "Speakers_by_session.csv"
speaker_df = pd.read_csv(file)

print(len(speaker_df))
print(speaker_df.columns)

In [ ]:
column_mapping = {
    'Year': 'year',
    'Session': 'session',
    'ISO Code': 'country',
    'Country': 'country_name',
    'Name of Person Speaking': 'speaker',
    'Post': 'position',
    'Language': 'language',
    'Notes': 'notes',
}

# select the new columns, i.e. not mapped to None
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

# select and rename columns
speaker_df = speaker_df[columns].rename(columns=column_mapping)

There is a wrong country code in one session:

In [ ]:
# bug in data: SLV is not Slovenia
speaker_df.loc[((speaker_df['session'] == 59) & (speaker_df['country_name'] == 'Slovenia')), 'country'] = 'SVN'

speaker_df[((speaker_df['session'] == 59) & (speaker_df['country_name'] == 'Slovenia'))]

Create index on session and country for join with speeches data frame:

In [ ]:
speaker_df.dropna(subset=['session', 'country'], inplace=True)
speaker_df.set_index(['session', 'country'], inplace=True, verify_integrity=True)

## Prepare Speeches Data

In [ ]:
file = "un-general-debates.csv"
df = pd.read_csv(file)

print(len(df))

Generate standardized country names with `pycountry`:

In [ ]:
import pycountry

def country_for_iso(iso):
    if iso == 'YUG':
        return 'Yugoslavia'
    elif iso == 'CSK':
        return 'Czechoslovakia'
    elif iso == 'DDR':
        return 'German Dem. Republic'
    elif iso == 'YDYE':
        return 'Democratic Yemen'
    elif iso == 'EU':
        return 'European Union'
    else:
        return pycountry.countries.get(alpha_3=iso).name

In [ ]:
# there are two codes for Yemen
# df.loc[df['country']=='YDYE', 'country'] = 'YEM'

df['country_name'] = df['country'].apply(country_for_iso)

In [ ]:
df.set_index(['session', 'country'], inplace=True, verify_integrity=True)
df.reset_index(inplace=True)

## Join Speeches and Speakers

In [ ]:
df = df.join(speaker_df, on=['session', 'country'], rsuffix='_spk')
df.columns
len(df)

In [ ]:
df = df[['session', 'year', 'country', 'country_name', 'speaker', 'position', 'text']]

df['session'] = df['session'].astype(int)
df['year'] = df['year'].astype(int)

df.sort_values(by=['year', 'session', 'country'], inplace=True)

## Save to csv

The file will be zipped (Pandas has a built-in compression and decompression feature).

In [ ]:
df.to_csv("un-general-debates-blueprint.csv.gz", index=False)

## Save to SQLite

Alternatively, we could also save this to a SQLite DB. 

In [ ]:
import sqlite3

db_path = "un-general-debates.db"

# con = sqlite3.connect(db_path)
# df.to_sql("un_debates", con, index=False, if_exists="replace")
# con.close()